# Libraries

In [ ]:
pip install yfinance yesg PyPortfolioOpt portfolio-backtest riskfolio-lib plotly scikit-learn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as opt
import yesg
#import plotly.express as px
#import seaborn as sns
import plotly.graph_objects as go
#from plotly.subplots import make_subplots
#import riskfolio as rp
import warnings
import requests
from io import BytesIO, StringIO
import pypfopt as pf
#warnings.filterwarnings("ignore")

# Equal risk portfolio weight optimization

In [ ]:
# Function to calculate the cost of transactions
def transaction_costs(w, w0, TC):
    return np.sum(TC * np.abs(w - w0)**2)

# Modified objective function to include transaction costs
def objective(w, cov_mat, w0, TC, factor_TC):
    # Original variance-based function
    s = 0
    for i in range(len(w)):
        for j in range(len(w)):
            s += (w[i] * (cov_mat @ w)[i] - w[j] * (cov_mat @ w)[j])**2
    # Add transaction costs as a penalty term
    return s + factor_TC * transaction_costs(w, w0, TC)
    
def rp_weights(df, w0= np.array([0.2, 0.2, 0.2, 0.2, 0.2]), TC = np.array([0.0005, 0.0005, 0.0005, 0.0005, 0.0005]), factor_TC = .0001):
    """
    Function to calculate the weights for the risk parity portfolio
    df: DataFrame with the ETF returns
    w0: Initial weights (current portfolio)
    TC: Transaction costs (5 bps)
    factor_TC: Scaling factor for transaction costs (tune this as needed)
    """
    cov_mat = df.cov().values  # Covariance matrix of ETF returns

    # Constraints: weights sum to 1 and are long-only
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum of weights = 1
                   {'type': 'ineq', 'fun': lambda w: w}]            # Weights must be positive (long only)

    # Optimize
    result = opt.minimize(objective, w0, args=(cov_mat, w0, TC, factor_TC), constraints=constraints, tol=1e-20)
    w_opt = result.x  # Optimized weights
    return w_opt

# Minimum variance portfolio weight optimization

In [ ]:
def min_var_weights(train_data):
    """Effectue l'optimisation Min-Var et retourne les poids optimaux."""
    try:
        mean_returns = train_data.mean()
        cov_matrix = train_data.cov()
        ef = pf.EfficientFrontier(mean_returns, cov_matrix)
        weights = ef.min_volatility()
        return np.array(list(weights.values()))
    except Exception as e:
        print(f"Erreur lors du calcul des poids du min-var portfolio : {e}")
        return None

# General functions

In [ ]:
def calculate_metrics(portfolio_returns, spy_returns, slice):
    """Calcule les métriques de performance pour une tranche donnée."""
    try:
        cumulative_return = float((1 + portfolio_returns).prod() - 1)
        cov_matrix = np.cov(portfolio_returns, spy_returns)
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        expected_annual_return = float(portfolio_returns.mean() * 252)
        volatility = float(portfolio_returns.std() * np.sqrt(i))
        sharpe_ratio = float(cumulative_return / volatility)
        treynor_ratio = float(cumulative_return / beta)


        return {
            "Slice": slice,
            "Expected Annual Return": expected_annual_return,
            "Volatility": volatility,
            "Sharpe Ratio": sharpe_ratio,
            "Treynor Ratio": treynor_ratio,
            "Cumulative Return": cumulative_return
        }
    except Exception as e:
        print(f"Erreur lors du calcul des métriques de la tranche {i} : {e}")
        return None

In [ ]:
def get_data(tickers, start_date, end_date):
    try:
        data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
        returns = data.pct_change().dropna()

        spy_data = yf.download('SPY', start=start_date, end=end_date)['Adj Close']
        spy_returns = spy_data.pct_change().dropna()
    except Exception as e:
        print(f"Erreur lors de la récupération des données : {e}")
        return
    return returns, spy_returns

In [ ]:
def convert_metrics(metrics_list):    
    try:
        metrics_df = pd.DataFrame(metrics_list)
        metrics_df = metrics_df.set_index("Slice")
    except KeyError as e:
        print(f"Erreur lors de la conversion en DataFrame : {e}")
        return
    except Exception as e:
        print(f"Erreur inattendue : {e}")
        return
    return metrics_df

In [ ]:
def backtest(i, j, k, start_date, end_date, tickers, strategy):

  # Désactiver les warnings pour les matrices non définies positives
    warnings.filterwarnings("ignore", message="You must convert self.cov to a positive definite matrix")
    try:
        # Vérification des paramètres d'entrée
        if j < i:
            raise ValueError("Le deuxième paramètre (j) doit être supérieur au premier (i).")
    except ValueError as e:
        print(f"Erreur d'entrée : {e}")
        return

    # Étape 1 : Récupération des données
    returns, spy_returns = get_data(tickers, start_date, end_date)

    # Étape 2 : Initialisation des variables
    try:
        train_data = returns.iloc[:j, :]
        test_data = returns.iloc[j:, :]
        spy_test_data = spy_returns.iloc[j:]
        metrics_list = []
        start_idx = 0
        weights_list = []
    except Exception as e:
        print(f"Erreur lors de l'initialisation des variables : {e}")
        return

    # Étape 3 : Optimisation RP pour chaque tranche
    try:
        while start_idx + i <= len(test_data) and start_idx + i <= len(spy_test_data) :
            # Etape 3 : Découpage des données de test
            slice_data = test_data.iloc[start_idx:start_idx + i]
            spy_slice_data = spy_test_data.iloc[start_idx:start_idx + i].to_numpy().flatten()

            # Etape 4 : Calcul des poids du portefeuille
            if strategy=="ERC":
                # Optimisation RP sur les données d'entraînement actuelles
                #port = rp.Portfolio(returns=train_data)
                #port.assets_stats(method_mu='hist', method_cov='hist')
                #weights = port.rp_optimization(model='Classic', rm='MV').to_numpy().flatten()
                weights = rp_weights(train_data)
                weights_list.append(weights)
                portfolio_returns = slice_data @ weights    # Calcul des rendements du portefeuille pour la tranche
            
            elif strategy=="Equal weight":
                weights = np.array([1/len(tickers)]*len(tickers))
                weights_list.append(weights)
                portfolio_returns = slice_data @ weights
            
            elif strategy=="Benchmark":
                portfolio_returns = spy_slice_data

            elif strategy=="Min-var":
                weights = min_var_weights(train_data)
                weights_list.append(weights)
                portfolio_returns = slice_data @ weights

            else:
                print("Stratégie non reconnue, veuillez choisir entre 'ERC', 'Equal weight', 'Benchmark' et 'Min-var'")
                return

            # Étape 5 : Calcul des métriques
            metrics_list.append(calculate_metrics(portfolio_returns, spy_slice_data, len(metrics_list) + 1))

            # Mise à jour des données d'entraînement
            train_data = pd.concat([train_data, test_data.iloc[start_idx:start_idx + k]]).iloc[-j:]
            start_idx += k

    except Exception as e:
        print(f"Erreur lors de l'optimisation ou du calcul des métriques : {e}")
        return

    # Étape 6 : Conversion en DataFrame
    metrics_df = convert_metrics(metrics_list)

    return metrics_df, weights_list

In [ ]:
tickers = ['SPHQ', 'IVE', 'SPYD', 'SPLV', 'SPMO']
start_date = '2018-01-01'
end_date = '2024-11-01'
i = 5
j = 22 
k = i
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, k, start_date, end_date, tickers, strategy)
    sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
    cumulative_returns[strategy] = metrics_df["Cumulative Return"]  

In [ ]:
def plot_data(data):
    # Create a figure
    fig = go.Figure()

    # Loop through each column in the DataFrame and add a trace for each
    for column in data.columns:
        fig.add_trace(go.Scatter(x=data.index, y=data[column], mode='lines', name=column))

    # Show the plot
    fig.show()

def box_plot_data(data):
    fig = go.Figure()

    for column in data.columns:
        fig.add_trace(go.Box(y=data[column], name=column))

    fig.update_layout(
        title="Box Plot",
        yaxis_title="Values",
        xaxis_title="Columns",
    )

    fig.show()

def plot_moving_average_data(data):
    # Apply a rolling mean (moving average) for smoothing
    smoothed_sr = data.rolling(window=30, min_periods=1).mean()  # 5-period rolling average

    fig = go.Figure()

    for column in smoothed_sr.columns:
        fig.add_trace(go.Scatter(x=smoothed_sr.index, y=smoothed_sr[column], mode='lines', name=column))

    # Update layout for better visualization
    fig.update_layout(
        title="Smoothed Trends of Each Column",
        xaxis_title="Index",
        yaxis_title="Smoothed Values",
    )

    fig.show()

In [ ]:
sharpe_ratios.describe()
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

In [ ]:
cumulative_returns.describe()
plot_data(cumulative_returns)
box_plot_data(cumulative_returns)
plot_moving_average_data(cumulative_returns)

# From ETFs weights to stocks weights

In [ ]:
def download_data(url):
    """
    Télécharge et retourne les données d'une URL, en gérant les fichiers Excel et CSV de manière appropriée.
    """
    if url.endswith('.xls') or url.endswith('.xlsx'):
        skip_rows = 4
        response = requests.get(url)
        response.raise_for_status()
        data = pd.read_excel(BytesIO(response.content), skiprows=skip_rows)
    else:
        skip_rows = 0
        response = requests.get(url)
        response.raise_for_status()
        csv_data = response.content.decode('utf-8')
        try:
            data = pd.read_csv(StringIO(csv_data), skiprows=skip_rows)
        except pd.errors.ParserError:
            try:
                data = pd.read_csv(StringIO(csv_data), delimiter=';', skiprows=skip_rows)
            except pd.errors.ParserError:
                data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)

    if not any('weight' in col.lower() for col in data.columns):
        data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)
    return data

def normalize_ticker(ticker):
    """
    Normalise le ticker en supprimant les espaces et les caractères inutiles.
    """
    if pd.isna(ticker):  # Vérifier si ticker est NaN
        return ''
    return ticker.replace('/', '').strip().upper()

def calculate_total_weights_per_etf(url, etf_weight):
    """
    Calcule le poids total pour chaque action d'un fichier CSV ou Excel.
    """
    data = download_data(url)
    weight_column = next((col for col in data.columns if 'weight' in col.lower()), None)
    if weight_column is None:
        print("Colonne 'Weight' ou 'Weight (%)' introuvable dans le fichier.")
        return None

    holding_ticker_column = next((col for col in data.columns if 'holding ticker' in col.lower()), None)
    ticker_column = next((col for col in data.columns if 'ticker' in col.lower()), None)
    if holding_ticker_column:
        ticker_column = holding_ticker_column
    elif not ticker_column:
        print("Aucune colonne 'Ticker' ou 'Holding Ticker' introuvable dans le fichier.")
        return None

    data['Weighted_Weight'] = data[weight_column] * etf_weight

    if 'Name' in data.columns:
        data = data[~data['Name'].isin(["US DOLLAR", "Cash/Receivables/Payables", "S+P500 EMINI FUT  DEC24", "CASH COLLATERAL MSFUT USD", "USD CASH", "SSI US GOV MONEY MARKET CLASS"])]

    # Normaliser les tickers
    data[ticker_column] = data[ticker_column].apply(normalize_ticker)
    
    total_weights = data.groupby(ticker_column)[['Name', 'Weighted_Weight']].sum().reset_index()
    total_weights.columns = ['Ticker', 'Name', 'Total_Weight']
    total_weights = total_weights[total_weights['Total_Weight'] > 0]
    return total_weights

def calculate_total_weights(weights):
    # Téléchargement et traitement des données pour chaque URL
    urls = [
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPHQ",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPLV",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPMO",
        "https://www.ishares.com/us/products/239728/ishares-sp-500-value-etf/1467271812596.ajax?fileType=csv&fileName=IVE_holdings&dataType=fund",
        "https://www.ssga.com/us/en/intermediary/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spyd.xlsx"
    ]

    final_data = pd.DataFrame()

    # Appliquer chaque URL et poids d'ETF
    for url, weight in zip(urls, weights):
        etf_data = calculate_total_weights_per_etf(url, weight)
        if etf_data is not None:
            final_data = pd.concat([final_data, etf_data])

    # Étape finale : Regrouper par ticker exact et sommer les poids
    final_data = final_data.groupby('Ticker', as_index=False)['Total_Weight'].sum()
    final_data['Total_Weight'] /= final_data['Total_Weight'].sum()

    # Trier les données en ordre décroissant de poids
    #final_data.sort_values(by='Total_Weight', ascending=False, inplace=True)

    # Vérification des doublons
    duplicate_tickers = final_data[final_data.duplicated(subset='Ticker', keep=False)]
    if not duplicate_tickers.empty:
        print("Doublons trouvés dans les tickers :")
        print(duplicate_tickers)
    #else:
    #    print("Aucun doublon trouvé dans les tickers.")

    # Affichage du résultat final
    #print(final_data.to_string(index=False))

    # Enregistrer dans un fichier CSV
    #final_data.to_csv("poids_actions_etf.csv", index=False)
    #print("Les données complètes ont été enregistrées dans 'poids_actions_etf.csv'.")

    final_data.columns = ['Symbol', 'Weights']
    final_data = final_data.set_index('Symbol')
    
    return final_data

#Stocks with several classes: fox and news corp

In [ ]:
stock_weights = calculate_total_weights(weights_list[-1])
stock_weights.sort_values(by='Weights', ascending=False)

# Constraints

- We do not take into account the worst stocks in terms of ESG scores
- Sector constraints (the s&p is already concentrated so it might be interesting to allow a difference with the sector wiehgts in the original s&p 500)
- Concentration limits (a limit for each stock)
- Liquidity constraints (use only stocks with volume higher than a threshold): not so interesting because all s&p 500 stocks are liquid
- no short allowed
- tracking error: determined with backtest

## ESG constraints

In [ ]:
def esg_constraints(stock_weights, quantile_threshold):
    """
    quantile_threshold: float between 0 and 1 to keep the best quantile_threshold quantile of the stocks
    stock_weights: DataFrame with the weights of the stocks in the portfolio
    """
    # get the s&p 500 tickers with their name, sector and sub-industry
    df_sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']]
    df_sp500 = df_sp500.set_index('Symbol') # set the index to be the symbol

    df_sp500.drop(index='GOOG', inplace=True) # drop the GOOG row because there is already a GOOGL row

    df_esg = df_sp500.copy()
    for ticker in df_sp500.index:
        try:
            df_esg.loc[ticker, "ESG Score"] = yesg.get_historic_esg(ticker).iloc[-1,0]
        except AttributeError:
            pass

    #drop all the stocks that have no ESG score
    df_esg.dropna(axis=0, inplace=True)

    #The best ESG score is 0
    #drop the worst stocks in terms of ESG score
    df_esg = df_esg[df_esg['ESG Score'] < df_esg['ESG Score'].quantile(quantile_threshold)]

    #drop all the stocks that have an ESG score above the threshold
    #threshold = 30
    #df_esg = df_esg[df_esg['ESG Score'] < threshold]

    df_esg = df_esg.merge(stock_weights, left_index=True, right_index=True)

    df_esg['Weights'] /= df_esg['Weights'].sum()

    return df_esg

In [ ]:
quantile_threshold = 0.9
df_esg = esg_constraints(stock_weights, quantile_threshold)
df_esg.sort_values(by='Weights', ascending=False)

## Sector constraints

In [ ]:
def sector_constraints(df, sector_threshold):
    """
    df: DataFrame following the same format as the one returned by esg_constraints
    sector_threshold: float between 0 and 1 that a sector cannot exceed in the portfolio
    """
    #We calculate the weights of the stocks in each sector
    df_weights_by_sector = df[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()
    
    #We create a list of the sectors that have a weight above the threshold
    sectors_above_threshold = df_weights_by_sector.loc[df_weights_by_sector['Weights'] > sector_threshold].index
    while len(sectors_above_threshold) > 0:
        for sector in sectors_above_threshold:
            #for each stock in the sector, we apply a factor such that the sum of all stocks in this sector is equal to the threshold
            df_sector = df[df['GICS Sector'] == sector]
            factor = sector_threshold / df_weights_by_sector.loc[sector, 'Weights']
            df.loc[df_sector.index, 'Weights'] = df_sector['Weights'] * factor
        
        #we increase the weights of the stocks in the other sectors to keep the sum of the weights equal to 1
        df.loc[~df['GICS Sector'].isin(sectors_above_threshold), "Weights"] /= df.loc[~df['GICS Sector'].isin(sectors_above_threshold), "Weights"].sum() / (1-sector_threshold*len(sectors_above_threshold))
        
        #Some sectors may now have a weight above the threshold so we repeat the process
        df_weights_by_sector = df[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()
        sectors_above_threshold = df_weights_by_sector.loc[df_weights_by_sector['Weights'] > sector_threshold].index

    return df

In [ ]:
sector_threshold = 0.17
weights_after_sector = sector_constraints(df_esg, sector_threshold)

#Check that none of the sectors have a weight above the threshold
#weights_after_sector[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()

#Check that the sum of the weights is equal to 1
#weights_after_sector[['GICS Sector', 'Weights']].groupby('GICS Sector').sum().sum()

weights_after_sector.sort_values(by='Weights', ascending=False)

,Weights
GICS Sector,
Communication Services,0.06
Consumer Discretionary,0.06
Financials,0.12
Health Care,0.17
Industrials,0.12
Information Technology,0.17
Materials,0.12
Real Estate,0.06
Utilities,0.12


## Stock constraints

In [ ]:
def stock_constraints(df, stock_threshold):
    """
    df: DataFrame following the same format as the one returned by sector_constraints
    stock_threshold: float between 0 and 1 that a stock cannot exceed in the portfolio
    """
    #We create a list of the stocks that have a weight above the threshold
    stocks_above_threshold = df.loc[df['Weights'] > stock_threshold].index
    while len(stocks_above_threshold) > 0:
        #we set the weights of the stocks above the threshold to the threshold
        df.loc[stocks_above_threshold, 'Weights'] = stock_threshold
        #we increase (proportionally) the weights of the other stocks to keep the sum of the weights equal to 1
        df.loc[~df.index.isin(stocks_above_threshold), "Weights"] /= df.loc[~df.index.isin(stocks_above_threshold), "Weights"].sum() / (1-stock_threshold*len(stocks_above_threshold))
        #Some stocks may now have a weight above the threshold so we repeat the process
        stocks_above_threshold = df.loc[df['Weights'] > stock_threshold].index

    return df

In [ ]:
#print("Max weight for a stock before stock constraints: ", weights_after_sector['Weights'].max())
stock_threshold = 0.05
weights_after_stock = stock_constraints(weights_after_sector, stock_threshold)
#print("Max weight for a stock after stock constraints: ", weights_after_stock['Weights'].max())

#Check that none of the stocks have a weight above the threshold
#weights_after_stock['Weights'].max()

#Check that the sum of the weights is equal to 1
#weights_after_stock['Weights'].sum()

weights_after_stock.sort_values(by='Weights', ascending=False)

### Liquidity constraints
We can limit the universe by only taking into account stocks with a volume higher than a threshold but I'm not sure it's necessary because all s&p 500 stocks are liquid.

### Tracking error constraints
Apart from using weights closer to the original s&p 500 weights, I don't really see how to reduce the tracking error. 

# References
### Implementations of equal risk contribution
- https://github.com/matthewgilbert/erc/blob/master/erc/erc.py
- https://github.com/mirca/riskparity.py (not used)
- https://thequantmba.wordpress.com/2016/12/14/risk-parityrisk-budgeting-portfolio-in-python/

### Papers
- [Paper of Maillard, Roncalli and Teiletche](http://thierry-roncalli.com/download/erc.pdf)
- [Slides of Maillard, Roncalli and Teiletche](http://www.thierry-roncalli.com/download/erc-slides.pdf)
- [Master's thesis of David Stefanovits](https://ethz.ch/content/dam/ethz/special-interest/math/risklab-dam/documents/walter-saxer-preis/ma-stefanovits.pdf)
